In [11]:
import os, warnings
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"
warnings.filterwarnings("ignore")

import tensorflow as tf
from tensorflow.keras import layers, models

BASE_PATH = "/kaggle/input"
DATASET_PATH = os.path.join(BASE_PATH, os.listdir(BASE_PATH)[0])
DATA_ROOT = os.path.join(DATASET_PATH, os.listdir(DATASET_PATH)[0])

IMG_SIZE = (128, 128)
BATCH_SIZE = 32
EPOCHS = 5
SEED = 42

train_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_ROOT,
    validation_split=0.2,
    subset="training",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_ROOT,
    validation_split=0.2,
    subset="validation",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

class_names = train_ds.class_names
num_classes = len(class_names)
print("Classes:", class_names)
print("Number of classes:", num_classes)

normalization_layer = layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))


train_ds = train_ds.apply(tf.data.experimental.ignore_errors())
val_ds = val_ds.apply(tf.data.experimental.ignore_errors())


model = models.Sequential([
    layers.Conv2D(32, (3,3), activation="relu", input_shape=(128,128,3)),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3), activation="relu"),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation="relu"),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dense(num_classes, activation="softmax")
])

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

# -------- Train --------
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS
)


loss, acc = model.evaluate(val_ds)
print("Validation Accuracy:", acc)

print("\n CNN trained successfully (corrupted images skipped).")

Found 12824 files belonging to 8 classes.
Using 10260 files for training.
Found 12824 files belonging to 8 classes.
Using 2564 files for validation.
Classes: ['asteroid', 'black hole', 'comet', 'constellation', 'galaxy', 'nebula', 'planet', 'star']
Number of classes: 8


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)              │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 8)              │         1,032 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,305,672 (12.61 MB)

 Trainable params: 3,305,672 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 93s 12s/step - accuracy: 0.1306 - loss: 2.2334 - val_accuracy: 0.3125 - val_loss: 1.9597
Epoch 2/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 146s 20s/step - accuracy: 0.2783 - loss: 1.9817 - val_accuracy: 0.2000 - val_loss: 1.9084
Epoch 3/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 92s 10s/step - accuracy: 0.2109 - loss: 1.8767 - val_accuracy: 0.2727 - val_loss: 1.9131
Epoch 4/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 89s 10s/step - accuracy: 0.2271 - loss: 1.9145 - val_accuracy: 0.2600 - val_loss: 1.8521
Epoch 5/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 85s 11s/step - accuracy: 0.2716 - loss: 1.8605 - val_accuracy: 0.4444 - val_loss: 1.6472
4/4 ━━━━━━━━━━━━━━━━━━━━ 17s 4s/step - accuracy: 0.2594 - loss: 1.8101
Validation Accuracy: 0.25

 CNN trained successfully (corrupted images skipped).
